In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from pathlib import Path
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *

Import the planets-dataset. A dataframe train_df is constructed from the the csv file containing the file names of images and the labels for each of the images. The labels are multicategory.

In [ ]:
train_path = Path('../input/planets-dataset/planet/planet')

train_df = pd.read_csv(train_path/'train_classes.csv')
train_df.head()

The Datablock API from fastai was used to prepare the training data testing data. The Dtablock object was passed to dataloaders object which is an iterable returning the mini batches of traing data and testing data as tuples. Each tuple has a batch of taining data i.e. image and the labe associated with it. The Datablock object also included the methods to transform and augment the training data. The item_tfms method transforms images itemwise using CPU and batch_tfms transforms images in batches using GPU.

In [ ]:
def get_x(r): return train_path/'train-jpg'/(r['image_name']+'.jpg')
def get_y(r): return r['tags'].split()

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                  splitter=RandomSplitter(seed=42),
                  get_x=get_x,
                  get_y=get_y,
                  item_tfms=Resize(224),
                  batch_tfms=[*aug_transforms(flip_vert=True, max_warp=0),
                             Normalize.from_stats(*imagenet_stats)])

In [ ]:
dls = dblock.dataloaders(train_df, bs=64)

In [ ]:
dls.show_batch(nrows=1, ncols=3)

The metrics of the model measure the performance of the model in the test data. Fastai by default uses multiclass accuracy for multilabel data. Here the f2 score is also defined to comply with this Kaggle competition rules.

In [ ]:
f2samples = FBetaMulti(beta=2, average='samples', thresh=0.5)
metrics = [partial(accuracy_multi, thresh=0.5), f2samples]
cbs = MixUp()

First, as a baseline model, let's train a resnet34 network. The lr_find() helps to find the optimum learning rate. It returns the learning rate at which loss is minimum (lr_min) and the learning rate at which the loss is decreasing at the fastest rate (lr_steep).   

In [ ]:
learn = cnn_learner(dls,resnet34, metrics=metrics, cbs=cbs)
lr_min, lr_steep = learn.lr_find()
lr_min, lr_steep

In [ ]:
learn.fine_tune(1, 3e-2)
lr_min, lr_steep = learn.lr_find()
lr_min, lr_steep

Here I am trying to get an optimum threshold for the metrics.

In [ ]:
preds, targets = learn.get_preds()
xs = torch.linspace(0.1, 0.9, 29)
accs = [accuracy_multi(preds, targets, thresh=i, sigmoid=False) for i in xs]
plt.plot(xs,accs)

The figure above suggests that the threshhold should be around 0.4.

In [ ]:
f2samples = FBetaMulti(beta=2, average='samples', thresh=0.4)
metrics = [partial(accuracy_multi, thresh=0.4), f2samples]

Now, lets train the model using fit_one_cycle() method of the learner. This method updates the weights of last layer of the pretrained resnet34 model and trains it predict the images in current dataset and keeps the earlier layers of the resnet34 model frozen to their pretrained values.
Then the learn.unfreeze() method is called to unfreeze those earlier layers. The learn.fit_one_cycle() method is called once again to  update t weights of all the layers of the cnn. Here, the lr_max parameter is passed to implement learning descriminitive learning rates. The lower value in the slice will be used for the earliest layer and the learning rates are multiplicatively scaled up to higher rates for later layers.

In [ ]:
learn.fit_one_cycle(3, 5e-4)
learn.unfreeze()
learn.fit_one_cycle(10, lr_max=slice(1e-5,1e-3))

In [ ]:
lr_min, lr_steep = learn.lr_find()
lr_min, lr_steep

In [ ]:
preds, targets = learn.get_preds()
xs = torch.linspace(0.1, 0.9, 29)
accs = [accuracy_multi(preds, targets, thresh=i, sigmoid=False) for i in xs]
plt.plot(xs,accs)

In [ ]:
f2samples = FBetaMulti(beta=2, average='samples', thresh=0.5)
metrics = [partial(accuracy_multi, thresh=0.5), f2samples]
learn.fine_tune(5, base_lr=1e-4)

In [ ]:
test_path = Path('../input/planets-dataset/planet/planet/test-jpg')
add_path = Path('../input/planets-dataset/test-jpg-additional/test-jpg-additional')

submiss_df = pd.read_csv(train_path/'sample_submission.csv')
test_path = (submiss_df['image_name']+'.jpg').apply(lambda x: test_path/x if x.startswith('test') else add_path/x)

test_dl = learn.dls.test_dl(test_path)
predictions = learn.get_preds(dl = test_dl)


In [ ]:
predlist = [' '.join (learn.dls.vocab[i]) for i in (predictions[0]>0.5)]
predlist

In [ ]:
submiss_df['tags'] = predlist


In [ ]:
out_path = Path('../kaggle/working')
submiss_df.to_csv(out_path/'amazon_resnet34.csv', index=False)

In [ ]:
#from fastai.callback.fp16 import *

#learn = cnn_learner(dls, resnet50, metrics=metrics).to_fp16()
#lr_min, lr_steep = learn.lr_find()
#lr_min, lr_steep

